In [2]:
from dotenv import load_dotenv
from openai import OpenAI,AsyncOpenAI
from agents import Agent, Runner, function_tool,Trace,input_guardrail,output_guardrail,FunctionTool,GuardrailFunctionOutput,OpenAIChatCompletionsModel,trace
from typing import Dict
from pydantic import BaseModel
import os

In [3]:
load_dotenv(override=True)

True

In [4]:

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [23]:
instruction1 = "You are Cricket Fanatic. You have extreme knowledge in cricket and includes records, statistics, player info and many other. Answer only to this context and nothing else .Kepp it elaboratoive.If the topic is outside cricket reply 'i am not aware of this topic and restricted to cricket only'"
instruction2 = "You are Bollywood Fanatic. You know all info about bollywood , celibrites, box office, release, actor and actress ifo etc etc. Answer only to this context and nothing else . Keep it elaboratoive. If the topic is outside bollywood reply 'i am not aware of this topic and restricted to bolllywood only'"
instruction3 = "You are Space and science fanatic. You have all info about universe, solar system blackholes, extraterestial,supernova,starts , planets galaxy and telescopes , rover and science missisions and exploration. Answer only in context and nothing else.Be elaborative.If the topic is outside space and science reply 'i am not aware of this topic and restricted to space and science only'"

In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [24]:
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

In [25]:
deepseek_model=OpenAIChatCompletionsModel(model='deepseek-chat',openai_client=deepseek_client)
# this is simalr to 2_lab2_practice.ipynb  where in 
'''
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"
response = deepseek.chat.completions.create(model=model_name, messages=messages)
'''

'\ndeepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")\nmodel_name = "deepseek-chat"\nresponse = deepseek.chat.completions.create(model=model_name, messages=messages)\n'

In [26]:
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [27]:
AgentPicker=Agent(name="Response Picker",model='gpt-4o-mini',instructions="""You are best response picker. you analyse response returned by LLMS/AGENTS OR tools and pick the best one as per context and the topic of interest given 
     just pick the best response from llms/tools/agents in context of the topic and area of expertise being asked. If the response is outside the topic u donty pick that . You are intelligent and 
        and dont pick any random things. You see which Agent/Tool had provided the topic answer and u chose that Agent/Tool response. u know some Agent/Tool may not provide the answer as it is outside their expertise. Choose the one which give the response for the 
            topic and output the response and reply with the best response. Dont answer by yourself any thing .Let the Agent/Tool Decide.
             Please Prefix at beigining of response the following (Only if you get Response related to the topic).
             channel_name:Channel1 if Bollywood related you picked or channel_name:Chhannel2 if cricket is picked and channel_name:Channel3 if space is picked . 
             after this with a single new line response should foolow. 
             Lastly give the description why u picked it in one sentence 
             Dont randomly select any channel if the response is outside their context .If none of the Agent provides suitable answer to topic respons with 'Topic oustsoide three channel(space,cricket,bollywood)
             . I am sorry i cant answer . foe example , if any question is asked for politics none of the tools will be able to answer , you should format the response as i stated  above  """
             )

In [28]:
BollyAgent=Agent(name="Bollywood DeepSeek Agent",instructions=instruction1,model=deepseek_model)
CricAgent=Agent(name="Cricket geninine agent",instructions=instruction2,model=gemini_model)
SpaceAgent=Agent(name="Space llama agent",instructions=instruction3,model=llama3_3_model)

In [29]:
tool1=BollyAgent.as_tool(tool_name="Bollywood",tool_description="all information about bollywood")
tool2=CricAgent.as_tool(tool_name="Cricket",tool_description="Info about cricket sport")
tool3=SpaceAgent.as_tool(tool_name="Space",tool_description="Info about space and science")
tool4=AgentPicker.as_tool(tool_name="ResponsePicker",tool_description="Pick the best response from the given options")



In [30]:
from IPython.display import display, Markdown

In [31]:
@function_tool
def extract_Chhanel_and_response(data:str):
    """Simple function to extract channel_name and response from tool call output"""
    if isinstance(data,dict) and 'text_output' in data:
        text_output=data['text_output']
    else:
        text_output=data
    
    lines = text_output.split('\n')
   
    channel = None
    response_lines = []
    
    for line in lines:
        # Extract channel name
        
        if 'channel_name' in line:
                channel = line.split(':')[1]

        # Extract Response
        elif line.strip() and not line.startswith('channel_name:'):
            response_lines.append(line.strip())
 
    
    response = '\n'.join(response_lines) if response_lines else ""

    if "Channel1" in channel:
        display(Markdown(f"## 🎬🎵🎥  **Bollywood Channel Selected**\n\n{response}"))
    elif "Channel2" in channel:
        display(Markdown(f"## 🏏🏆🏟️ **Cricket Channel Selected**\n\n{response}"))
    elif "Channel3" in channel:
        display(Markdown(f"## ⭐🪐🌍  **space Channel Selected**\n\n{response}"))
        

    
    return channel, response

In [32]:
@function_tool
def content_display(channel: str, response: str):
    '''
    display contents visually apealing

    Args:
        channel: The channel to display, such as "Channel1" or "Channel2 or Channel3".
        response: The content to display on the channel.
    '''
    if "Channel1" in channel:
        display(Markdown(f"## 🎬🎵🎥  **Bollywood Channel Selected**\n\n{response}"))
    elif "Channel2" in channel:
        display(Markdown(f"## 🏏🏆🏟️ **Cricket Channel Selected**\n\n{response}"))
    elif "Channel3" in channel:
        display(Markdown(f"## ⭐🪐🌍  **space Channel Selected**\n\n{response}"))

In [33]:
final_handoff_instruction="""you are an Content Displayer . u recieve the response and channel_name from the tool extract_Chhanel_and_response. and u generate the content in markdown which is visually apealing 
.You also add icon related to the subject  along with the display of content .You also Take care of pargraph spacing and make the conttent display very visually apealing in markdown format"""

In [34]:
display_content_tool=[content_display]
display_content_agent=Agent(name="Content Displayer",
                            model='gpt-4o-mini',
                            instructions=final_handoff_instruction,
                            handoff_description="Display content in visually apealing markdown format",
                            tools=display_content_tool)
display_handoff=[display_content_agent]

In [35]:
content_generator_nd_response_tool=[tool1,tool2,tool3,tool4]

In [36]:
Tinstruction= """You are a content agent .Based on subject given to you , you leverage the below tools for the topic or subject 
            tools=Bollywood ,Cricket ,Space , ResponsePicker and extract_Chhanel_and_response
            DO NOT answer by yourself and DO NOT display by urself. always use tools at your disposal to answer the question. You are just a guide      
            Follow the below instruction properly 
               1. Whatever question is asked by the user you need to pass to the 3 tools for Content genreation: Bollywood ,Cricket and  Space tools
               2. The response provided by the above tools should be passed to ResponsePicker tool and let the ResponsePicker tool decided the response to be picked. The tool should also pick the channel name along with content
               3. Next use the tool extract_Chhanel_and_response for extracting channel_name and response taking the input from Response Picker tool.Ensure that channel_name is picked .
                   Also ensure only one channel is selected and that too to the subject concerned. If there is a relation between channels  you should  judge and pick one and only one response.
               4. Finally Handoff for displaying: pass the response and channel_name to display_content_agent. the display_content_agent will take care of displaying visually appealing content
                critical rules:
                    1. Always use tool for performing tasks.
                    2. Use must handoff by passing the response and channel_name to display_content_agent 
                    3. if there is an issue with any tool or handoff ,output the same and elaborate the issue for debugging"""

In [37]:
content_manager=Agent(
                        name="Content Manager",
                        model="gpt-4o-mini",
                        instructions=Tinstruction,
                        handoffs=display_handoff,
                        tools=content_generator_nd_response_tool
)

In [38]:
message="what is supernova"
with trace("ContentCreateorr"):
    result=await Runner.run(content_manager,message)

## ⭐🪐🌍  **space Channel Selected**

# 🌌 What is a Supernova?

A **supernova** is an incredibly **powerful** and **rare** astronomical event in which a star explodes, releasing a massive amount of **energy** and **elements** into space. It is one of the most **violent** and **spectacular** events in the universe, outshining an entire galaxy of stars and expelling a vast amount of gas, dust, and heavy elements into the surrounding interstellar medium.

## Types of Supernovae

There are two main types of supernovae:

- **Type I**: These occur when a white dwarf star in a binary system accumulates material from a companion star, eventually reaching a critical mass that triggers a **thermonuclear explosion**.

- **Type II**: These occur when a massive star (at least 8-10 times the mass of the sun) runs out of fuel and collapses under its own gravity, leading to a massive explosion.

## Characteristics of Supernovae

1. **Energy Output**: The explosion can be so powerful that it can be seen from millions of light-years away, briefly outshining an entire galaxy.
2. **Element Expulsion**: Heavy elements like **iron**, **nickel**, and **gold** are forged in the star's core during the explosion and are then dispersed into space.
3. **Role in the Universe**: Supernovae play a crucial role in the evolution of the universe by helping to distribute heavy elements, seeding the formation of new stars and planets, and regulating galaxy growth.
4. **Scientific Importance**: They provide unique opportunities to study extreme physics and gain insights into the fundamental laws governing matter and energy.

## Examples of Notable Supernovae
- **SN 1987A**: A Type II supernova that occurred in the Large Magellanic Cloud, a satellite galaxy of the Milky Way.
- **SN 1006**: A Type Ia supernova that was visible from Earth for several months.
- **Tycho's Supernova**: A Type Ia supernova observed in 1572 by Danish astronomer Tycho Brahe.

Overall, supernovae are awe-inspiring events that offer profound insights into the extreme physics of the universe, shaping the cosmic landscape in which we exist.

In [39]:
from typing import List
class InappropriateContentCheck(BaseModel):
    has_inappropriate_content: bool
    inappropriate_words_found: List[str]

inappropriate_guardrail_agent = Agent(
    name="Inappropriate Content Check",
    instructions="Check if the user's message contains inappropriate words or topics like hate, violence, spam, nudity ,porn related,nudity related ,ass ,dick, illegal, harmful, dangerous, toxic, abuse, harassment.",
    output_type=InappropriateContentCheck,
    model="gpt-4o-mini"
)

In [40]:
@input_guardrail
async def guardrail_against_inappropriate_content(ctx, agent, message):
    # Run the guardrail agent to check the message
    result = await Runner.run(inappropriate_guardrail_agent, message, context=ctx.context)
    
    # Get the result from Pydantic model
    has_inappropriate = result.final_output.has_inappropriate_content
    
    if has_inappropriate:
        print("🚫 BLOCKING content")
    # Return GuardrailFunctionOutput
    return GuardrailFunctionOutput(
        output_info={"inappropriate_words": result.final_output.inappropriate_words_found},
        tripwire_triggered=has_inappropriate  # True = block, False = allow
    )

    # ctx.context contains the conversation history
    # ctx.agent is the BollyAgent/CricAgent/SpaceAgent
    # message is the user's input

In [41]:
NonResponsisbleContentManger=Agent(                        name="Content Manager",
                        model="gpt-4o-mini",
                        instructions=Tinstruction,
                        handoffs=display_handoff,
                        tools=content_generator_nd_response_tool,
                        input_guardrails=[guardrail_against_inappropriate_content])

In [42]:
message="no of hits of akshay kumar"
with trace("non_responspible content"):
    result=await Runner.run(NonResponsisbleContentManger,message)

## 🎬🎵🎥  **Bollywood Channel Selected**

# 🌟 Akshay Kumar: The Versatile Star of Bollywood

---

### 🎬 About Akshay Kumar

Akshay Kumar, born **Rajiv Hari Om Bhatia** on **September 9, 1967**, is one of Bollywood's most prolific and bankable actors. Known for his **versatility**, **discipline**, and **dedication** to his craft, he has successfully transitioned from being an action hero to a comedic star, while also addressing socially relevant subjects.

---

### 📈 Key Highlights:

- **Early Career & Action Hero Image:**  
  Akshay started his Bollywood journey in the early 1990s, primarily known for action-packed roles in films like **"Khiladi"**, **"Mohra"**, and **"Sabse Bada Khiladi"**. His martial arts background gave him an edge in these roles.

- **Transition to Comedy:**  
  He reinvented himself with comedic roles in movies such as **"Hera Pheri"**, **"Mujhse Shaadi Karogi"**, and **"Singh Is Kinng"**.

- **Socially Relevant Films:**  
  Recently, he has focused on films that tackle social issues, including **"Toilet: Ek Prem Katha"** (sanitation) and **"Pad Man"** (menstrual hygiene).

- **Box Office Power:**  
  Akshay Kumar is consistently ranked among the highest-paid actors in Bollywood, known for his commercially successful films.

- **Awards & Recognition:**  
  He has won numerous awards, including two **National Film Awards** for Best Actor and several **Filmfare Awards**.

---

### 🎥 Notable Films (Acros Genres):

- **Action:** Khiladi series, Mohra, Waqt Hamara Hai  
- **Comedy:** Hera Pheri, Housefull series  
- **Drama/Social:** Airlift, Toilet: Ek Prem Katha, Gold  
- **Romantic:** Dhadkan, Namastey London  

---

### 🌟 Why He is So Popular:

- **Versatility:**  
  Convinces in action, comedy, and drama.

- **Discipline:**  
  Known for a rigorous fitness routine and professional work ethics.

- **Relatability:**  
  Often portrays characters that resonate with the common man.

- **Patriotic Image:**  
  His films reflect a sense of patriotism and social responsibility.

---

### 🎉 Conclusion

Akshay Kumar continues to be a dominant force in Bollywood, constantly evolving with his roles while maintaining his status as a top star.

In [43]:
message="hate speech from cricket celebrites"
with trace("non_responspible content"):
    result=await Runner.run(NonResponsisbleContentManger,message)

    # check trace :
   # {
  #"has_inappropriate_content": true,
  #"inappropriate_words_found": [
   # "nude"
 ## ]
#}


🚫 BLOCKING content


InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [132]:
# 1. Create Pydantic model for output validation
class OutputQualityCheck(BaseModel):
    is_response_blank: bool
    response_quality_score: int  # 1-10 scale
    issues_found: str

# 2. Create the output guardrail agent
output_guardrail_agent = Agent(
    name="Output Quality Check",
    instructions="""Check if the agent's response is appropriate  and relevant to the Tools tpic of concern and high quality .
    Rate quality from 1-10 and list any issues. Consider: relevance to topic, helpfulness, appropriateness, completeness.
     If its outside the topic what the Bollywood tool or space tool  or cricket tool  , The response should be blank 
     and set response_quality_score to 0.
     """,
    output_type=OutputQualityCheck,
    model="gpt-4o-mini"
)

# 3. Create the output guardrail function
@output_guardrail
async def guardrail_against_poor_output(ctx, agent, response):
    # Run the guardrail agent to check the response
    result = await Runner.run(output_guardrail_agent, response, context=ctx.context)
    
    # Get the results from Pydantic model
    quality_score = result.final_output.response_quality_score
    is_appropriate = result.final_output.is_response_blank
   
    issues = result.final_output.issues_found
    
    # Block if quality is too low or inappropriate
    should_block = not is_appropriate or quality_score < 5
    #print(should_block) 
    should_block = quality_score < 1

    if not should_block:
        print("✅ ALLOWING content") 
    else :
        display(Markdown("❌ BLOCKED :The user query is outside my scope to answer. Please restric your question to bOllywood,cricket or Space & science related only"))
    
    return GuardrailFunctionOutput(
        output_info={
            "quality_score": quality_score,
            "issues_found": issues,
            "agent_name": agent.name
        },
        tripwire_triggered=should_block
    )



In [133]:
ResponsisbleContentManger=Agent(                        name="Content Manager",
                        model="gpt-4o-mini",
                        instructions=Tinstruction,
                        handoffs=display_handoff,
                        tools=content_generator_nd_response_tool,
                        input_guardrails=[guardrail_against_inappropriate_content],
                        output_guardrails=[guardrail_against_poor_output])

In [134]:
message="why is donald trump imposing tarrifs ?"
with trace("non_responspible content"):
    result=await Runner.run(ResponsisbleContentManger,message)

❌ BLOCKED :The user query is outside my scope to answer. Please restric your question to bOllywood,cricket or Space & science related only

OutputGuardrailTripwireTriggered: Guardrail OutputGuardrail triggered tripwire

In [95]:
message="virat kohli  ?"
with trace("responspible content"):
    result=await Runner.run(ResponsisbleContentManger,message)

## 🏏🏆🏟️ **Cricket Channel Selected**

# 🏏 **Virat Kohli: The Phenomenon of Cricket**

Virat Kohli is not just a player; he is a phenomenon. He is widely regarded as one of the greatest batsmen in the history of the sport. A modern-day legend, his career is defined by incredible consistency, an insatiable hunger for runs, and a fierce competitive spirit.

---

## 🏆 **Highlights of His Career**
- **Full Name:** Virat Kohli  
- **Born:** November 5, 1988, in Delhi, India  
- **Role:** Top-Order Batsman (Right-Handed)  
- **Current Teams:** India (International), Royal Challengers Bangalore (IPL)  
- **Nicknames:** King Kohli, The Chase Master, Run Machine  

---

## 📈 **Career Statistics**  
* **Total Runs:** Over **26,000** international runs  
* **Centuries:** **80** international centuries  
  - **Test Centuries:** 29  
  - **ODI Centuries:** 50 *(Highest ever)*  
  - **T20I Centuries:** 1  

#### 🔍 **Format Breakdown**  
- **Test Cricket:**  
  - **Matches:** 113 | **Runs:** 8,848 | **Average:** 49.16  
  - **Highest Score:** 254  

- **ODI Cricket:**  
  - **Matches:** 292 | **Runs:** 13,848 | **Average:** 58.68  

- **T20I Cricket:**  
  - **Matches:** 117 | **Runs:** 4,037 | **Average:** 51.75  

---

## 🔥 **The Chase Master**  
Kohli's ability to chase down targets in limited-overs cricket is legendary.  
* **Average in Successful ODI Chases:** **90.90**  
* **Centuries in Chases:** **26**  

---

## 🎖️ **Leadership**  
He served as the captain of India across all formats from 2017 to 2022, leaving a legacy of fitness, aggression, and a never-say-die attitude.

---

## ⭐ **Legacy & Impact**  
Virat Kohli's impact transcends statistics, inspiring millions of young athletes around the globe.

### In Summary  
**Virat Kohli** is the quintessential modern batsman—a blend of classical technique, explosive power, supreme fitness, and an unyielding will to win. His career is a masterclass in consistency and excellence, making him a vital force in cricket.

#Just see another way

In [54]:
from pydantic import BaseModel, Field, validator

class SimpleInputGuard(BaseModel):
    """Simple Pydantic model to block inappropriate words"""
    user_query: str = Field(..., description="User's question")
    
    @validator('user_query')
    def block_inappropriate_words(cls, v):
        """Block queries with inappropriate words"""
        
        # List of inappropriate words to block
        inappropriate_words = [
            "hate", "violence", "spam", "scam", "fraud", "illegal",
            "harmful", "dangerous", "toxic", "abuse", "harassment"
        ]
        
        # Check if any inappropriate word is in the query
        query_lower = v.lower()
        found_bad_words = [word for word in inappropriate_words if word in query_lower]
        
        if found_bad_words:
            raise ValueError(f"Blocked: Contains inappropriate words: {found_bad_words}")
        
        return v

# Simple guardrail function
def simple_input_guardrail(user_query: str) -> str:
    """Simple input guardrail using Pydantic"""
    try:
        # Validate using Pydantic
        validated = SimpleInputGuard(user_query=user_query)
        return validated.user_query
    except Exception as e:
        return str(e)

# Test it
test_queries = [
    "What's the latest Bollywood movie?",  # ✅ Good
    "Tell me about hate speech",           # ❌ Bad
    "Who is the best cricket player?",     # ✅ Good
    "How to scam people?",                 # ❌ Bad
]

for query in test_queries:
    result = simple_input_guardrail(query)
    if "Blocked:" in result:
        print(f"❌ BLOCKED: {query}")
    else:
        print(f"✅ ALLOWED: {query}")

✅ ALLOWED: What's the latest Bollywood movie?
❌ BLOCKED: Tell me about hate speech
✅ ALLOWED: Who is the best cricket player?
❌ BLOCKED: How to scam people?


/var/folders/nc/2bjvclms3k13mrldbxsjxx780000gn/T/ipykernel_1640/4286803232.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('user_query')


In [ ]:
# Add this after your agent definitions (around cell 9)

@input_guardrail
def validate_user_input(user_query: str) -> str:
    """Validate and filter user input"""
    
    # Block inappropriate content
    inappropriate_words = ["hate", "violence", "spam", "scam","nude","porn","ass"]
    if any(word in user_query.lower() for word in inappropriate_words):
        return "I cannot process this type of request. Please ask about Bollywood, Cricket, or Space/Science topics."
    
    # Ensure query is not too long
    if len(user_query) > 500:
        return "Your question is too long. Please keep it under 500 characters."
    
    # Ensure query is not empty
    if not user_query.strip():
        return "Please provide a valid question."
    
    return user_query

# Apply to your agents
BollyAgent.add_input_guardrail(validate_user_input)
CricAgent.add_input_guardrail(validate_user_input)
SpaceAgent.add_input_guardrail(validate_user_input)